Auto-reload any function that are configured on the back-end.

In [1]:
%load_ext autoreload
%autoreload 2

Main application path:

In [3]:
import os
from os import walk
# main app directory
# main_app_path = os.path.dirname(os.path.dirname(os.getcwd()))
main_app_path = os.path.dirname(os.path.dirname(os.getcwd()))
print(main_app_path)
# changes the current working directory to the given path
# os.chdir(main_app_path)

/home/iotlab_user/Desktop/PYTHON/LAB/dev_tasks


In [4]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)


In [5]:
# find path
import os
from pathlib import Path
# cnn model
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [6]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

In [7]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [8]:
# load a dataset group, such as train or test
def load_dataset_group(main_app_path, group, prefix=''):
    filepath = main_app_path + "/" + "datasets/" + prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [9]:
# load the dataset, returns train and test X and y elements
def load_dataset(main_app_path, prefix=''):
    # load all train
    trainX, trainy = load_dataset_group(main_app_path, 'train', prefix + 'UCI HAR Dataset/')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [10]:
trainX, trainy, testX, testy = load_dataset(main_app_path)

FileNotFoundError: [Errno 2] File UCI HAR Dataset/train/y_train.txt does not exist: 'UCI HAR Dataset/train/y_train.txt'

In [ ]:
trainX.shape

In [ ]:
trainX[0].shape

In [ ]:
trainX[0]

In [ ]:
df = pd.DataFrame(data=trainX[0])

In [ ]:
trainy[0]

In [ ]:
df.head()

In [ ]:
df.plot(figsize=(20,10), linewidth=2, fontsize=20)
plt.xlabel('Sample', fontsize=20);
plt.ylabel('Axes', fontsize=20);

In [ ]:
# standardize data
def scale_data(trainX, testX, standardize):
    # remove overlap
    cut = int(trainX.shape[1] / 2)
    longX = trainX[:, -cut:, :]
    # flatten windows
    longX = longX.reshape((longX.shape[0] * longX.shape[1], longX.shape[2]))
    # flatten train and test
    flatTrainX = trainX.reshape((trainX.shape[0] * trainX.shape[1], trainX.shape[2]))
    flatTestX = testX.reshape((testX.shape[0] * testX.shape[1], testX.shape[2]))
    # standardize
    if standardize:
        s = StandardScaler()
        # fit on training data
        s.fit(longX)
        # apply to training and test data
        longX = s.transform(longX)
        flatTrainX = s.transform(flatTrainX)
        flatTestX = s.transform(flatTestX)
    # reshape
    flatTrainX = flatTrainX.reshape((trainX.shape))
    flatTestX = flatTestX.reshape((testX.shape))
    return flatTrainX, flatTestX

In [ ]:
trainX_scaled, testX_scaled = scale_data(trainX, testX, standardize=True)

In [ ]:
df_scaled = pd.DataFrame(data=trainX_scaled[0])

In [ ]:
df_scaled.head()

In [ ]:
df_scaled.plot(figsize=(20,10), linewidth=2, fontsize=20)
plt.xlabel('Sample', fontsize=20);
plt.ylabel('Axes', fontsize=20);

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, param):
    verbose, epochs, batch_size = 0, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # scale data
    trainX, testX = scale_data(trainX, testX, param)
    model = keras.models.Sequential()
    model.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.MaxPooling1D(pool_size=2))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [ ]:
# summarize scores
def summarize_results(scores, params):
    print(scores, params)
    # summarize mean and standard deviation
    for i in range(len(scores)):
        m, s = mean(scores[i]), std(scores[i])
        print('Param=%s: %.3f%% (+/-%.3f)' % (params[i], m, s))
    # boxplot of scores
    pyplot.boxplot(scores, labels=params)
    pyplot.savefig('exp_cnn_standardize.png')

In [ ]:
# run an experiment
def run_experiment(params, repeats=10):
    # load data
    trainX, trainy, testX, testy = load_dataset()
    # test each parameter
    all_scores = list()
    for p in params:
        # repeat experiment
        scores = list()
        for r in range(repeats):
            score = evaluate_model(trainX, trainy, testX, testy, p)
            score = score * 100.0
            print('>p=%s #%d: %.3f' % (p, r+1, score))
            scores.append(score)
        all_scores.append(scores)
    # summarize results
    summarize_results(all_scores, params)

In [ ]:
# run the experiment
n_params = [False, True]
run_experiment(n_params)